# Fixed Effects Models 

Fixed effects model is a statistical model in which the model parameters are fixed or non-random quantities. This is in contrast to random effects models and mixed models in which all or some of the model parameters are considered as random variables. (Wikipedia)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2018-08-27'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [3]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,1,1,1,0.012658
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,1,1,1,0.032500
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0,1,0,0.012107
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0,0,0,0.000000
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0,0,1,-0.021531


In [32]:
dataset.tail()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2018-08-21,19.980000,20.420000,19.860001,20.400000,20.400000,55629000,1,1,1,0.021021
2018-08-22,20.280001,20.920000,20.209999,20.900000,20.900000,62002700,1,1,1,0.024510
2018-08-23,21.190001,22.320000,21.139999,22.290001,22.290001,113444100,1,1,1,0.066507
2018-08-24,22.910000,24.000000,22.670000,23.980000,23.980000,164328200,1,1,1,0.075819
2018-08-27,24.940001,27.299999,24.629999,25.260000,25.260000,325058400,0,0,0,0.053378


In [33]:
dataset.shape

(1171, 10)

In [27]:
# Use more than 1 independent and dependent variables. (Minimum 2 variables for each)
X = np.array(dataset[['High', 'Low']])
y = np.array(dataset[['Open', 'Adj Close']])

In [29]:
from linearmodels.panel import PanelOLS

mod = PanelOLS(y, X, entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res.params)

Exog    0.999409
Name: parameter, dtype: float64


In [31]:
res.predict()

fitted_values
entity   time               
entity.0 0          3.997637
         1          4.177531
         2          4.247490
         3          4.257484
         4          4.227501
         5          4.197519
         6          4.197519
         7          4.297460
         8          4.597283
         9          4.507336
         10         4.347431
         11         4.337437
         12         3.817744
         13         3.677826
         14         3.607868
         15         3.487939
         16         3.567891
         17         3.557897
         18         3.517921
         19         3.467950
         20         3.437968
         21         3.427974
         22         3.368009
         23         3.457956
         24         3.497933
         25         3.637850
         26         3.747785
         27         3.727797
         28         3.707809
         29         3.717803
...                      ...
entity.1 1141      16.470266
         1142      16.540224
         1143      16.540224
         1144      16.430290
         1145      15.890608
         1146      16.100485
         1147      15.710715
         1148      16.820059
         1149      18.299184
         1150      19.298593
         1151      18.259209
         1152      18.309179
         1153      17.989368
         1154      18.359150
         1155      18.449095
         1156      19.068730
         1157      19.248624
         1158      19.068730
         1159      18.838866
         1160      19.108708
         1161      19.618404
         1162      19.188660
         1163      19.238630
         1164      18.718937
         1165      19.338571
         1166      19.848270
         1167      20.198062
         1168      21.127512
         1169      22.656610
         1170      24.615451

[2342 rows x 1 columns]